# To create a database with Education and Incarceration Data

In [1]:
import numpy as np 
import pandas as pd 

# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func


## Read in the Incarceration Data

In [2]:
crime = pd.read_csv('../Data/incarceration_data/ucr_by_state.csv')
prisoners_in_custody = pd.read_csv('../Data/incarceration_data/prison_custody_by_state.csv')
incarceration = pd.read_csv('../Data/incarceration_data/crime_and_incarceration_by_state.csv')
vera_incarceration_trends = pd.read_csv('../Data/incarceration_data/incarceration_trends.csv')



#### Preview Incarceration CSVs


In [3]:
display('BJS: Crime Rates by Crime Type State and Year')
display(crime.head(10))
display(crime.columns)

display('BJS: In Custody Counts by State and Year')
display(prisoners_in_custody.head(10))
display(prisoners_in_custody.columns)

display('BJS: In Custody Counts, Crime Rates by Type, By State and Year')
display(incarceration.head(10))
display(incarceration.columns)

display('Vera Justice: In Custody breakout by Demographic and Prison')
display(vera_incarceration_trends.head(10))
display(vera_incarceration_trends.columns)



'BJS: Crime Rates by Crime Type State and Year'

,jurisdiction,year,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,Alaska,2001,0,0,"633,630","3,735",39,501,NaN,514,"2,681","23,160","3,847","16,695","2,618"
1,Alaska,2002,0,0,"641,482","3,627",33,511,NaN,489,"2,594","24,118","3,908","17,739","2,471"
2,Alaska,2003,0,0,"648,280","3,877",39,605,NaN,446,"2,787","24,386","3,874","18,051","2,461"
3,Alaska,2004,0,0,"657,755","4,159",37,558,NaN,447,"3,117","22,172","3,773","16,159","2,240"
4,Alaska,2005,0,0,"663,253","4,194",32,538,NaN,537,"3,087","23,975","4,131","17,249","2,595"
5,Alaska,2006,0,0,"670,053","4,610",36,512,NaN,600,"3,462","23,975","4,155","17,284","2,536"
6,Alaska,2007,0,0,"683,478","4,520",43,545,NaN,581,"3,351","23,096","3,734","16,929","2,433"
7,Alaska,2008,0,0,"686,293","4,475",27,447,NaN,645,"3,356","20,090","3,237","15,225","1,628"
8,Alaska,2009,0,0,"698,473","4,424",22,513,NaN,654,"3,235","20,538","3,600","15,251","1,687"
9,Alaska,2010,0,0,"714,146","4,537",31,533,NaN,594,"3,379","20,259","3,105","15,535","1,619"


Index(['jurisdiction', 'year', 'crime_reporting_change', 'crimes_estimated',
       'state_population', 'violent_crime_total', 'murder_manslaughter',
       'rape_legacy', 'rape_revised', 'robbery', 'agg_assault',
       'property_crime_total', 'burglary', 'larceny', 'vehicle_theft'],
      dtype='object')

'BJS: In Custody Counts by State and Year'

,jurisdiction,includes_jails,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,Federal,0,"149,852","158,216","168,144","177,600","186,364","190,844","197,285","198,414","205,087","206,968","214,774","216,915","214,989","209,561","195,622","188,311"
1,Alabama,0,"24,741","25,100","27,614","25,635","24,315","24,103","25,253","25,363","27,241","27,345","26,813","26,768","26,825","26,145","25,212","23,745"
2,Alaska,1,"4,570","4,351","4,472","4,534","4,798","5,052","5,151","4,997","5,472","5,369","6,216","6,308","5,081","6,323","5,247","4,378"
3,Arizona,0,"27,710","29,359","31,084","32,384","33,345","35,752","37,700","39,455","40,544","40,130","39,949","40,013","41,031","42,136","42,204","42,248"
4,Arkansas,0,"11,489","11,849","12,068","12,577","12,455","12,854","13,275","13,135","13,338","14,192","14,090","14,043","14,295","15,250","15,784","15,833"
5,California,0,"157,142","159,695","161,785","163,939","168,035","172,298","171,444","171,085","168,830","162,821","147,578","132,935","134,339","134,430","127,815","129,416"
6,Colorado,0,"17,278","18,636","19,450","19,655","21,063","22,051","22,666","23,211","22,725","22,752","21,862","20,328","20,184","20,469","19,959","19,486"
7,Connecticut,1,"17,507","18,716","18,023","17,993","17,928","18,902","19,438","19,905","18,938","18,629","17,877","17,164","17,319","16,814","16,024","15,040"
8,Delaware,1,"6,841","6,637","6,630","6,753","6,788","7,021","7,110","6,944","6,581","6,378","6,546","6,730","6,798","6,730","6,437","6,334"
9,Florida,0,"72,404","75,210","79,594","83,165","86,559","90,377","95,187","99,878","101,373","102,070","100,861","99,835","100,940","100,873","99,485","98,010"


...

,jurisdiction,includes_jails,year,prisoner_count,crime_reporting_change,crimes_estimated,state_population,violent_crime_total,murder_manslaughter,rape_legacy,rape_revised,robbery,agg_assault,property_crime_total,burglary,larceny,vehicle_theft
0,FEDERAL,False,2001,149852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ALABAMA,False,2001,24741,False,False,4468912.0,19582.0,379.0,1369.0,NaN,5584.0,12250.0,173253.0,40642.0,119992.0,12619.0
2,ALASKA,True,2001,4570,False,False,633630.0,3735.0,39.0,501.0,NaN,514.0,2681.0,23160.0,3847.0,16695.0,2618.0
3,ARIZONA,False,2001,27710,False,False,5306966.0,28675.0,400.0,1518.0,NaN,8868.0,17889.0,293874.0,54821.0,186850.0,52203.0
4,ARKANSAS,False,2001,11489,False,False,2694698.0,12190.0,148.0,892.0,NaN,2181.0,8969.0,99106.0,22196.0,69590.0,7320.0
5,CALIFORNIA,False,2001,157142,False,False,34600463.0,212867.0,2206.0,9960.0,NaN,64614.0,136087.0,1134189.0,232273.0,697739.0,204177.0
6,COLORADO,False,2001,17278,False,False,4430989.0,15492.0,158.0,1930.0,NaN,3555.0,9849.0,170887.0,28533.0,121360.0,20994.0
7,CONNECTICUT,True,2001,17507,False,False,3434602.0,11492.0,105.0,639.0,NaN,4183.0,6565.0,95299.0,17159.0,65762.0,12378.0
8,DELAWARE,True,2001,6841,False,False,796599.0,4868.0,23.0,420.0,NaN,1156.0,3269.0,27399.0,5144.0,19476.0,2779.0
9,FLORIDA,False,2001,72404,False,False,16373330.0,130713.0,874.0,6641.0,NaN,32867.0,90331.0,782517.0,176052.0,516548.0,89917.0


Index(['jurisdiction', 'includes_jails', 'year', 'prisoner_count',
       'crime_reporting_change', 'crimes_estimated', 'state_population',
       'violent_crime_total', 'murder_manslaughter', 'rape_legacy',
       'rape_revised', 'robbery', 'agg_assault', 'property_crime_total',
       'burglary', 'larceny', 'vehicle_theft'],
      dtype='object')

'Vera Justice: In Custody breakout by Demographic and Prison'

,yfips,year,fips,state,county_name,total_pop,total_pop_15to64,female_pop_15to64,male_pop_15to64,asian_pop_15to64,...,violent_crime,property_crime,murder_crime,rape_crime,robbery_crime,agr_assault_crime,burglary_crime,larceny_crime,mv_theft_crime,arson_crime
0,197001001,1970,1001,AL,Autauga County,24661,14154,7293,6861,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,197101001,1971,1001,AL,Autauga County,25503,14765,7585,7180,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,197201001,1972,1001,AL,Autauga County,27156,15939,8168,7771,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,197301001,1973,1001,AL,Autauga County,28453,16906,8651,8255,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,197401001,1974,1001,AL,Autauga County,29261,17578,8992,8586,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,197501001,1975,1001,AL,Autauga County,29716,18007,9210,8797,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,197601001,1976,1001,AL,Autauga County,29892,18476,9447,9029,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,197701001,1977,1001,AL,Autauga County,30457,19068,9757,9311,NaN,...,42.0,655.0,5.0,7.0,12.0,18.0,183.0,422.0,50.0,0.0
8,197801001,1978,1001,AL,Autauga County,30879,19566,10013,9553,NaN,...,21.0,651.0,2.0,4.0,7.0,8.0,146.0,451.0,54.0,0.0
9,197901001,1979,1001,AL,Autauga County,32050,20523,10505,10018,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Index(['yfips', 'year', 'fips', 'state', 'county_name', 'total_pop',
       'total_pop_15to64', 'female_pop_15to64', 'male_pop_15to64',
       'asian_pop_15to64', 'black_pop_15to64', 'latino_pop_15to64',
       'native_pop_15to64', 'other_pop_15to64', 'white_pop_15to64',
       'urbanicity', 'region', 'division', 'commuting_zone', 'metro_area',
       'land_area', 'total_jail_adm', 'total_jail_adm_dcrp',
       'female_jail_adm_dcrp', 'male_jail_adm_dcrp', 'total_jail_pop',
       'female_jail_pop', 'male_jail_pop', 'asian_jail_pop', 'black_jail_pop',
       'latino_jail_pop', 'native_jail_pop', 'white_jail_pop',
       'total_jail_pretrial', 'female_jail_pretrial', 'male_jail_pretrial',
       'jail_from_state_prison', 'jail_from_other_state_prison',
       'jail_from_state_jail', 'jail_from_other_state_jail', 'jail_from_fed',
       'jail_from_ice', 'total_jail_pop_dcrp', 'female_jail_pop_dcrp',
       'male_jail_pop_dcrp', 'total_prison_pop', 'female_prison_pop',
       'male_prison

#### Drop Columns that will be excluded from report (exclude Federal)

In [4]:
incarceration_trim = incarceration.drop(columns=['includes_jails','crime_reporting_change','crimes_estimated',
                                        'murder_manslaughter', 'rape_legacy', 'rape_revised',
                                        'robbery', 'agg_assault', 'burglary', 'larceny', 'vehicle_theft'])

incarceration_df = incarceration_trim.loc[incarceration_trim['jurisdiction'] != 'FEDERAL']
incarceration_format = incarceration_df.rename(columns={'jurisdiction': 'state'})
incarceration_format.columns = map(str.upper, incarceration_format.columns)
incarceration_format.head()



,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
1,ALABAMA,2001,24741,4468912.0,19582.0,173253.0
2,ALASKA,2001,4570,633630.0,3735.0,23160.0
3,ARIZONA,2001,27710,5306966.0,28675.0,293874.0
4,ARKANSAS,2001,11489,2694698.0,12190.0,99106.0
5,CALIFORNIA,2001,157142,34600463.0,212867.0,1134189.0


#### Estimate Missing NY Incarceration values to repair 2015 NaN

In [5]:
incarceration_ny_fix = incarceration_format[incarceration_format['STATE'] == 'NEW YORK']
ny_14 = incarceration_ny_fix[(incarceration_ny_fix.STATE == 'NEW YORK') & (incarceration_ny_fix.YEAR == 2014)]
ny_16 = incarceration_ny_fix[(incarceration_ny_fix.STATE == 'NEW YORK') & (incarceration_ny_fix.YEAR == 2016)]

ny_15_state_pop = (ny_14.iloc[0]['STATE_POPULATION'].astype(int) 
                   + ny_16.iloc[0]['STATE_POPULATION'].astype(int))/2
                        
ny_15_vc_tot = (ny_14.iloc[0]['VIOLENT_CRIME_TOTAL'].astype(int) + 
                ny_16.iloc[0]['VIOLENT_CRIME_TOTAL'].astype(int))/2
                      
ny_15_pc_tot = (ny_14.iloc[0]['PROPERTY_CRIME_TOTAL'].astype(int) + ny_16.iloc[0]['PROPERTY_CRIME_TOTAL'].astype(int))/2
print(ny_15_state_pop, ny_15_vc_tot, ny_15_pc_tot)



19791256.5 74856.5 322253.0


#### Update NY Incarceration NaNs to estimated values  //  Check Repair

In [6]:
values = {'STATE_POPULATION': ny_15_state_pop, 
            'VIOLENT_CRIME_TOTAL': ny_15_vc_tot, 
            'PROPERTY_CRIME_TOTAL': ny_15_pc_tot
            }
incarceration_format.fillna(value=values, inplace=True)

display(incarceration_format.head())

ny_edit_check = round(incarceration_format[(incarceration_format.STATE == 'NEW YORK') & 
                                                  (incarceration_format.YEAR == 2015)],0)
display(ny_edit_check)



,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
1,ALABAMA,2001,24741,4468912.0,19582.0,173253.0
2,ALASKA,2001,4570,633630.0,3735.0,23160.0
3,ARIZONA,2001,27710,5306966.0,28675.0,293874.0
4,ARKANSAS,2001,11489,2694698.0,12190.0,99106.0
5,CALIFORNIA,2001,157142,34600463.0,212867.0,1134189.0


,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
746,NEW YORK,2015,51485,19791256.0,74856.0,322253.0


In [7]:
incarceration_format['STATE'] = incarceration_format['STATE'].str.replace(' ', '_', regex=True)
# incarceration_order = incarceration_format[['YEAR','STATE','PRISONER_COUNT', 'STATE_POPULATION', 
  #                                           'VIOLENT_CRIME_TOTAL', 'PROPERTY_CRIME_TOTAL']]
incarceration_sort = incarceration_format.sort_values(['STATE','YEAR'])
incarceration_index = incarceration_sort.reset_index()
incarceration_format = incarceration_index.drop(columns=['index'])
incarceration_format.head()



,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
0,ALABAMA,2001,24741,4468912.0,19582.0,173253.0
1,ALABAMA,2002,25100,4478896.0,19931.0,180400.0
2,ALABAMA,2003,27614,4503726.0,19331.0,182241.0
3,ALABAMA,2004,25635,4525375.0,19324.0,182340.0
4,ALABAMA,2005,24315,4548327.0,19678.0,177393.0


#### Export cleaned Incarceration Data for Snapshot // Preview Dataframe

In [8]:
incarceration_report = round(incarceration_format,0)
incarceration_report.to_csv('../Reports/incarceration_report.csv')
incarceration_report



,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
0,ALABAMA,2001,24741,4468912.0,19582.0,173253.0
1,ALABAMA,2002,25100,4478896.0,19931.0,180400.0
2,ALABAMA,2003,27614,4503726.0,19331.0,182241.0
3,ALABAMA,2004,25635,4525375.0,19324.0,182340.0
4,ALABAMA,2005,24315,4548327.0,19678.0,177393.0
5,ALABAMA,2006,24103,4599030.0,19553.0,181249.0
6,ALABAMA,2007,25253,4627851.0,20775.0,184082.0
7,ALABAMA,2008,25363,4661900.0,21110.0,190416.0
8,ALABAMA,2009,27241,4708708.0,21194.0,178007.0
9,ALABAMA,2010,27345,4785401.0,18363.0,168828.0


## Read in the Vera Justice Incarceration Data
* Provides breakout by race

In [9]:
vera_incarceration_trends.columns



Index(['yfips', 'year', 'fips', 'state', 'county_name', 'total_pop',
       'total_pop_15to64', 'female_pop_15to64', 'male_pop_15to64',
       'asian_pop_15to64', 'black_pop_15to64', 'latino_pop_15to64',
       'native_pop_15to64', 'other_pop_15to64', 'white_pop_15to64',
       'urbanicity', 'region', 'division', 'commuting_zone', 'metro_area',
       'land_area', 'total_jail_adm', 'total_jail_adm_dcrp',
       'female_jail_adm_dcrp', 'male_jail_adm_dcrp', 'total_jail_pop',
       'female_jail_pop', 'male_jail_pop', 'asian_jail_pop', 'black_jail_pop',
       'latino_jail_pop', 'native_jail_pop', 'white_jail_pop',
       'total_jail_pretrial', 'female_jail_pretrial', 'male_jail_pretrial',
       'jail_from_state_prison', 'jail_from_other_state_prison',
       'jail_from_state_jail', 'jail_from_other_state_jail', 'jail_from_fed',
       'jail_from_ice', 'total_jail_pop_dcrp', 'female_jail_pop_dcrp',
       'male_jail_pop_dcrp', 'total_prison_pop', 'female_prison_pop',
       'male_prison

In [10]:
vera_incarceration_trends_trim = vera_incarceration_trends.drop(columns=[
'yfips','fips', 'county_name', 'total_pop_15to64', 'female_pop_15to64', 'male_pop_15to64',
'asian_pop_15to64', 'black_pop_15to64', 'latino_pop_15to64', 'native_pop_15to64', 'other_pop_15to64', 
'division', 'jail_from_state_prison', 'jail_from_other_state_prison', 'jail_from_state_jail', 
'total_jail_pretrial', 'female_jail_pretrial', 'male_jail_pretrial', 'female_jail_pop', 'male_jail_pop',
'jail_from_other_state_jail', 'jail_from_fed','jail_from_ice', 'urbanicity', 'commuting_zone','female_prison_pop', 'male_prison_pop', 
'metro_area', 'land_area', 'total_jail_adm_dcrp', 'female_jail_adm_dcrp', 'male_jail_adm_dcrp', 'total_jail_pop_dcrp', 'female_jail_pop_dcrp', 'male_jail_pop_dcrp',
'white_pop_15to64', 'total_prison_adm', 'female_prison_adm', 'male_prison_adm', 'asian_prison_adm', 'black_prison_adm',
'latino_prison_adm', 'native_prison_adm', 'other_prison_adm', 'white_prison_adm', 'index_crime', 'violent_crime',
'property_crime', 'murder_crime', 'rape_crime', 'robbery_crime', 'agr_assault_crime', 'burglary_crime', 'larceny_crime',
'mv_theft_crime', 'arson_crime', 'num_facilites', 'num_employees', 'confined_pop', 'capacity', 'ucr_population', 
                                    ])

vera_incarceration_trends_df = vera_incarceration_trends_trim.loc[vera_incarceration_trends_trim['year'] > 2000 ]
vit_drop = vera_incarceration_trends_df.dropna()
vit_group = vit_drop.groupby(by=['year','state']).sum()
vera_incarceration = vit_group.reset_index()
vera_incarceration


,year,state,total_pop,total_jail_adm,total_jail_pop,asian_jail_pop,black_jail_pop,latino_jail_pop,native_jail_pop,white_jail_pop,total_prison_pop,asian_prison_pop,black_prison_pop,latino_prison_pop,native_prison_pop,other_prison_pop,white_prison_pop
0,2001,AL,3996710,179699.647300,10340.999820,16.833333,5439.863337,177.995555,13.939394,4225.141375,21358.000000,0.000000,13501.000000,0.000000,0.000000,65.000000,7792.000000
1,2001,AZ,4576816,186845.201000,9855.333360,45.170455,1208.485714,3522.676160,459.666670,4660.942900,21622.000000,43.000000,3430.000000,8038.000000,1012.000000,329.000000,8770.000000
2,2001,CA,32337616,938169.673400,69962.666670,1376.116670,18139.666667,28176.499963,506.333863,21155.000377,148838.000000,1300.000000,47430.000000,53002.000000,1245.000000,5196.000000,40382.000000
3,2001,CO,3144101,124752.698200,6505.626326,70.333333,1537.476194,1966.371915,72.666666,3760.281521,12902.000000,138.000000,3741.000000,3369.000000,284.000000,0.000000,5370.000000
4,2001,FL,7821618,385573.639300,24130.000020,40.550000,10925.095233,3259.246550,20.518638,10667.619120,33836.000000,0.000000,18085.000000,4401.000000,41.000000,63.000000,11246.000000
5,2001,GA,2798484,197376.202290,12741.840270,24.148921,7471.119535,1180.465213,11.809635,3879.834150,17467.000000,35.000000,11517.000000,180.000000,10.000000,29.000000,5696.000000
6,2001,IA,771386,32361.567600,1084.600208,17.000000,214.666667,159.000003,25.000000,761.705451,2545.000000,27.090909,572.363636,153.363636,76.909091,13.000000,1702.272727
7,2001,IL,7879618,190341.373860,12927.702457,45.285716,8784.113732,1528.047141,10.125000,2634.408796,32501.000000,67.000000,22976.000000,3703.000000,45.000000,3.000000,5707.000000
8,2001,KY,1351189,79287.452460,4222.977971,5.000000,649.157185,115.013635,0.392738,3397.682639,3924.000000,0.000000,585.000000,12.000000,0.000000,4.000000,3323.000000
9,2001,MD,2434256,78648.811800,6712.333300,20.333333,4706.999940,236.634921,2.000000,1914.333370,14302.000000,22.000000,11657.000000,0.000000,8.000000,109.000000,2506.000000


In [11]:
vera_incarceration['state'] = vera_incarceration['state'].map({'AL':'ALABAMA',
'AK':'ALASKA', 'AZ':'ARIZONA','AR':'ARKANSAS','CA':'CALIFORNIA',
'CO':'COLORADO','CT':'CONNECTICUT','DE':'DELAWARE','FL':'FLORIDA',
'GA':'GEORGIA','HI':'HAWAII','ID':'IDAHO','IL':'ILLINOIS','IN':'INDIANA',
'IA':'IOWA','KS':'KANSAS','KY':'KENTUCKY','LA':'LOUISIANA','ME':'MAINE',
'MD':'MARYLAND','MA':'MASSACHUSETTS','MI':'MICHIGAN','MN':'MINNESOTA',
'MS':'MISSISSIPPI','MO':'MISSOURI','MT':'MONTANA','NE':'NEBRASKA',
'NV':'NEVADA','NH':'NEW HAMPSHIRE','NJ':'NEW JERSEY','NM':'NEW MEXICO',
'NY':'NEW YORK','NC':'NORTH CAROLINA','ND':'NORTH DAKOTA','OH':'OHIO',
'OK':'OKLAHOMA','OR':'OREGON','PA':'PENNSYLVANIA','RI':'RHODE ISLAND',
'SC':'SOUTH CAROLINA','SD':'SOUTH DAKOTA','TN':'TENNESSEE','TX':'TEXAS',
'UT':'UTAH','VT':'VERMONT','VA':'VIRGINIA','WA':'WASHINGTON',
'WV':'WEST VIRGINIA','WI':'WISCONSIN','WY':'WYOMING',})
vera_incarceration.head()


,year,state,total_pop,total_jail_adm,total_jail_pop,asian_jail_pop,black_jail_pop,latino_jail_pop,native_jail_pop,white_jail_pop,total_prison_pop,asian_prison_pop,black_prison_pop,latino_prison_pop,native_prison_pop,other_prison_pop,white_prison_pop
0,2001,ALABAMA,3996710,179699.6473,10340.999820,16.833333,5439.863337,177.995555,13.939394,4225.141375,21358.0,0.0,13501.0,0.0,0.0,65.0,7792.0
1,2001,ARIZONA,4576816,186845.2010,9855.333360,45.170455,1208.485714,3522.676160,459.666670,4660.942900,21622.0,43.0,3430.0,8038.0,1012.0,329.0,8770.0
2,2001,CALIFORNIA,32337616,938169.6734,69962.666670,1376.116670,18139.666667,28176.499963,506.333863,21155.000377,148838.0,1300.0,47430.0,53002.0,1245.0,5196.0,40382.0
3,2001,COLORADO,3144101,124752.6982,6505.626326,70.333333,1537.476194,1966.371915,72.666666,3760.281521,12902.0,138.0,3741.0,3369.0,284.0,0.0,5370.0
4,2001,FLORIDA,7821618,385573.6393,24130.000020,40.550000,10925.095233,3259.246550,20.518638,10667.619120,33836.0,0.0,18085.0,4401.0,41.0,63.0,11246.0


In [12]:
vera_incarceration['state'] = vera_incarceration['state'].str.replace(' ', '_', regex=True)
vera_incarceration.columns = map(str.upper, vera_incarceration.columns)
vera_incarceration = round(vera_incarceration, 0)
vera_incarceration.columns

Index(['YEAR', 'STATE', 'TOTAL_POP', 'TOTAL_JAIL_ADM', 'TOTAL_JAIL_POP',
       'ASIAN_JAIL_POP', 'BLACK_JAIL_POP', 'LATINO_JAIL_POP',
       'NATIVE_JAIL_POP', 'WHITE_JAIL_POP', 'TOTAL_PRISON_POP',
       'ASIAN_PRISON_POP', 'BLACK_PRISON_POP', 'LATINO_PRISON_POP',
       'NATIVE_PRISON_POP', 'OTHER_PRISON_POP', 'WHITE_PRISON_POP'],
      dtype='object')

In [13]:
vera_incarceration['ASIAN_JAIL_POP'] = ((vera_incarceration['ASIAN_JAIL_POP'] / 
                                            vera_incarceration['TOTAL_JAIL_POP']) * 100)

vera_incarceration['BLACK_JAIL_POP'] = ((vera_incarceration['BLACK_JAIL_POP'] / 
                                            vera_incarceration['TOTAL_JAIL_POP']) * 100)

vera_incarceration['LATINO_JAIL_POP'] = ((vera_incarceration['LATINO_JAIL_POP'] / 
                                            vera_incarceration['TOTAL_JAIL_POP']) * 100)

vera_incarceration['NATIVE_JAIL_POP'] = ((vera_incarceration['NATIVE_JAIL_POP'] / 
                                            vera_incarceration['TOTAL_JAIL_POP']) * 100)

vera_incarceration['WHITE_JAIL_POP'] = ((vera_incarceration['WHITE_JAIL_POP'] / 
                                            vera_incarceration['TOTAL_JAIL_POP']) * 100)

vera_incarceration['ASIAN_PRISON_POP'] = ((vera_incarceration['ASIAN_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)

vera_incarceration['BLACK_PRISON_POP'] = ((vera_incarceration['BLACK_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)

vera_incarceration['LATINO_PRISON_POP'] = ((vera_incarceration['LATINO_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)

vera_incarceration['NATIVE_PRISON_POP'] = ((vera_incarceration['NATIVE_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)

vera_incarceration['OTHER_PRISON_POP'] = ((vera_incarceration['OTHER_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)

vera_incarceration['WHITE_PRISON_POP'] = ((vera_incarceration['WHITE_PRISON_POP'] / 
                                            vera_incarceration['TOTAL_PRISON_POP']) * 100)


In [14]:
vera_incarceration_final = vera_incarceration.drop(columns=['TOTAL_POP','TOTAL_JAIL_POP','TOTAL_PRISON_POP'])
vir_sort = vera_incarceration_final.sort_values(['STATE', 'YEAR'])
vir_index = vir_sort.reset_index()
vera_incarceration_reset = vir_index.drop(columns=['index'])
vera_incarceration_report = vera_incarceration_reset[[
       'STATE','YEAR', 'TOTAL_JAIL_ADM', 
       'ASIAN_JAIL_POP', 'BLACK_JAIL_POP', 'LATINO_JAIL_POP',
       'NATIVE_JAIL_POP', 'WHITE_JAIL_POP',
       'ASIAN_PRISON_POP', 'BLACK_PRISON_POP', 'LATINO_PRISON_POP',
       'NATIVE_PRISON_POP', 'OTHER_PRISON_POP', 'WHITE_PRISON_POP']]
vera_incarceration_report.head()


,STATE,YEAR,TOTAL_JAIL_ADM,ASIAN_JAIL_POP,BLACK_JAIL_POP,LATINO_JAIL_POP,NATIVE_JAIL_POP,WHITE_JAIL_POP,ASIAN_PRISON_POP,BLACK_PRISON_POP,LATINO_PRISON_POP,NATIVE_PRISON_POP,OTHER_PRISON_POP,WHITE_PRISON_POP
0,ALABAMA,2001,179700.0,0.164394,52.606131,1.721304,0.135383,40.856784,0.0,63.212848,0.0,0.0,0.304336,36.482817
1,ALABAMA,2002,221681.0,0.297122,51.764160,2.293408,0.129991,44.113278,0.0,62.229226,0.0,0.0,0.221946,37.548828
2,ALABAMA,2003,158972.0,0.224417,41.303860,5.262567,0.112208,48.417864,0.0,53.034932,0.0,0.0,0.212512,46.752557
3,ALABAMA,2004,197394.0,0.297680,42.527202,4.034079,0.164237,47.207966,0.0,54.822695,0.0,0.0,0.153664,45.023641
4,ALABAMA,2005,194139.0,0.278124,42.476264,3.462166,0.143857,48.288098,0.0,53.184792,0.0,0.0,0.166646,46.648562


In [15]:
vera_incarceration_report.to_csv('../Reports/vera_incarceration_report.csv')
vera_incarceration_report.head()


,STATE,YEAR,TOTAL_JAIL_ADM,ASIAN_JAIL_POP,BLACK_JAIL_POP,LATINO_JAIL_POP,NATIVE_JAIL_POP,WHITE_JAIL_POP,ASIAN_PRISON_POP,BLACK_PRISON_POP,LATINO_PRISON_POP,NATIVE_PRISON_POP,OTHER_PRISON_POP,WHITE_PRISON_POP
0,ALABAMA,2001,179700.0,0.164394,52.606131,1.721304,0.135383,40.856784,0.0,63.212848,0.0,0.0,0.304336,36.482817
1,ALABAMA,2002,221681.0,0.297122,51.764160,2.293408,0.129991,44.113278,0.0,62.229226,0.0,0.0,0.221946,37.548828
2,ALABAMA,2003,158972.0,0.224417,41.303860,5.262567,0.112208,48.417864,0.0,53.034932,0.0,0.0,0.212512,46.752557
3,ALABAMA,2004,197394.0,0.297680,42.527202,4.034079,0.164237,47.207966,0.0,54.822695,0.0,0.0,0.153664,45.023641
4,ALABAMA,2005,194139.0,0.278124,42.476264,3.462166,0.143857,48.288098,0.0,53.184792,0.0,0.0,0.166646,46.648562


In [16]:
STATE = vera_incarceration_report['STATE'].nunique()
display(f'We have summary breakouts of Race for {STATE} states')
display(f'But we dont have coverage for every year, so we are just gonna keep this to ourselves')
display(vera_incarceration_report['STATE'].value_counts())
display(f'Still worth it for other projects')


'We have summary breakouts of Race for 38 states'

'But we dont have coverage for every year, so we are just gonna keep this to ourselves'

ALABAMA           15
NEW_JERSEY        15
CALIFORNIA        15
NORTH_CAROLINA    15
KENTUCKY          15
COLORADO          15
NEW_YORK          15
GEORGIA           15
FLORIDA           15
MINNESOTA         15
TEXAS             15
MAINE             15
SOUTH_CAROLINA    15
PENNSYLVANIA      15
MICHIGAN          15
MISSISSIPPI       15
NEBRASKA          15
MISSOURI          15
UTAH              15
NORTH_DAKOTA      15
TENNESSEE         15
IOWA              15
NEVADA            15
MARYLAND          15
ARIZONA           15
WEST_VIRGINIA     14
WISCONSIN         14
OKLAHOMA          14
WASHINGTON        14
SOUTH_DAKOTA      14
OREGON            14
INDIANA           14
ILLINOIS          14
VIRGINIA          13
WYOMING           10
NEW_HAMPSHIRE     10
MASSACHUSETTS      8
OHIO               6
Name: STATE, dtype: int64

'Still worth it for other projects'

## Read in the Education Data

In [17]:
education_file = "../Data/education_data/states_all.csv"
education_df = pd.read_csv(education_file)
display(education_df.head())
display(education_df.columns)


,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859713
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


Index(['PRIMARY_KEY', 'STATE', 'YEAR', 'ENROLL', 'TOTAL_REVENUE',
       'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE',
       'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE',
       'SUPPORT_SERVICES_EXPENDITURE', 'OTHER_EXPENDITURE',
       'CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G',
       'GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G',
       'GRADES_9_12_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE',
       'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'],
      dtype='object')

#### Notes on what some of the columns mean: 

* <b>Academic Achievement -  National Assessment of Educational Progress (NAEP)</b><br>
A breakdown of student performance as assessed by the corresponding exams (math and reading, grades 4 and 8).<br><br>

* <b>AVG_MATH_4_SCORE:</b> The state's average score for fourth graders taking the NAEP math exam.
* <b>AVG_MATH_8_SCORE:</b> The state's average score for eight graders taking the NAEP math exam.
* <b>AVG_READING_4_SCORE:</b> The state's average score for fourth graders taking the NAEP reading exam.
* <b>AVG_READING_8_SCORE:</b> The state's average score for eighth graders taking the NAEP reading exam.

In [18]:
# Create a filtered dataframe from specific columns

education_cols = ['STATE', 'YEAR', 'TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE', 'GRADES_4_G', 'GRADES_8_G', 'GRADES_ALL_G', 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE', 'AVG_READING_4_SCORE', 'AVG_READING_8_SCORE']
education_transformed = education_df[education_cols].copy()

education_transformed.head()

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,GRADES_4_G,GRADES_8_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,ALABAMA,1992,2653798.0,1481703.0,57948.0,58025.0,676174.0,208.327876,252.187522,207.963517,NaN
1,ALASKA,1992,972488.0,498362.0,9748.0,8789.0,112335.0,NaN,NaN,NaN,258.859713
2,ARIZONA,1992,3401580.0,1435908.0,55433.0,49081.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,ARKANSAS,1992,1743022.0,964323.0,34632.0,36011.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,CALIFORNIA,1992,27138832.0,14358922.0,418418.0,363296.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [19]:
# Rename the column headers
education_transformed = education_transformed.rename(columns={"GRADES_4_G": "4TH_ENROLLED",
                                                         "GRADES_8_G": "8TH_ENROLLED",
                                                         "GRADES_ALL_G": "TOTAL_ENROLLMENT",
                                                         "AVG_MATH_4_SCORE": "4_AVG_MATH_SCORE",
                                                         "AVG_MATH_8_SCORE": "8_AVG_MATH_SCORE",
                                                         "AVG_READING_4_SCORE": "4_AVG_RDG_SCORE",
                                                         "AVG_READING_8_SCORE": "8_AVG_RDG_SCORE"})
education_transformed.head()

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
0,ALABAMA,1992,2653798.0,1481703.0,57948.0,58025.0,676174.0,208.327876,252.187522,207.963517,NaN
1,ALASKA,1992,972488.0,498362.0,9748.0,8789.0,112335.0,NaN,NaN,NaN,258.859713
2,ARIZONA,1992,3401580.0,1435908.0,55433.0,49081.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,ARKANSAS,1992,1743022.0,964323.0,34632.0,36011.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,CALIFORNIA,1992,27138832.0,14358922.0,418418.0,363296.0,4717112.0,208.398961,260.892247,196.764414,NaN


In [20]:
# Filter by year - greater than the year 2000

education_year = education_transformed[education_transformed['YEAR'] > 2000]
education_year.head()


,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
459,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
460,ALASKA,2001,1402839.0,701053.0,10718.0,10702.0,124254.0,NaN,NaN,NaN,NaN
461,ARIZONA,2001,5858385.0,2538936.0,74270.0,69643.0,841750.0,NaN,NaN,NaN,NaN
462,ARKANSAS,2001,2951677.0,1616883.0,35216.0,35372.0,413278.0,NaN,NaN,NaN,NaN
463,CALIFORNIA,2001,52082540.0,26641811.0,485301.0,461133.0,5717489.0,NaN,NaN,NaN,NaN


In [21]:
# Filter by year - less than the year 2017

education_years = education_year[education_year['YEAR'] < 2017]
education_years.head()



,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
459,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
460,ALASKA,2001,1402839.0,701053.0,10718.0,10702.0,124254.0,NaN,NaN,NaN,NaN
461,ARIZONA,2001,5858385.0,2538936.0,74270.0,69643.0,841750.0,NaN,NaN,NaN,NaN
462,ARKANSAS,2001,2951677.0,1616883.0,35216.0,35372.0,413278.0,NaN,NaN,NaN,NaN
463,CALIFORNIA,2001,52082540.0,26641811.0,485301.0,461133.0,5717489.0,NaN,NaN,NaN,NaN


## A Quick view on NaNs and Missing Data
* Not all states evaluate their students on the NAEP rubric
* Finance and Enrollment data not available for all years for all Territories (States mostly seem fine)

In [22]:
display(education_years.count())
display(education_years['STATE'].value_counts().value_counts())


STATE                      927
YEAR                       927
TOTAL_EXPENDITURE          821
INSTRUCTION_EXPENDITURE    821
4TH_ENROLLED               848
8TH_ENROLLED               848
TOTAL_ENROLLMENT           825
4_AVG_MATH_SCORE           358
8_AVG_MATH_SCORE           358
4_AVG_RDG_SCORE            358
8_AVG_RDG_SCORE            358
dtype: int64

16    49
1      5
2      4
3      3
15     2
14     2
6      2
19     1
18     1
10     1
4      1
Name: STATE, dtype: int64

In [23]:
# Drop rows that are not in the US
#nonUS = ['GUAM', 'PUERTO_RICO', 'AMERICAN_SAMOA','VIRGIN_ISLANDS',
 #       'NORTHERN_MARIANAS','DOD_DOMESTIC','DOD_OVERSEAS',
  #      'BUREAU_OF_INDIAN_AFFAIRS','NORTHERN_MARIANA_ISLANDS','BI',
   #     'DD','DOD_-_DOMESTIC','BIE','DOD_-_OVERSEAS','BUREAU_OF_INDIAN_EDUCATION',
    #    'DEPARTMENT_OF_DEFENSE','DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY',
     #   'BUREAU_OF_INDIAN_EDUCATIO','U.S._VIRGIN_ISLANDS','DOD_(OVERSEAS_AND_DOMESTIC_COMBINED)'
      #  ]

updated_ed_df = education_years.loc[(education_years['STATE'] != 'GUAM') &
                                    (education_years['STATE'] != 'PUERTO_RICO') &
                                    (education_years['STATE'] != 'AMERICAN_SAMOA') &
                                    (education_years['STATE'] != 'VIRGIN_ISLANDS') &
                                    (education_years['STATE'] != 'NORTHERN_MARIANAS') &
                                    (education_years['STATE'] != 'DOD_DOMESTIC') &
                                    (education_years['STATE'] != 'DOD_OVERSEAS') &
                                    (education_years['STATE'] != 'BUREAU_OF_INDIAN_AFFAIRS') &
                                    (education_years['STATE'] != 'NORTHERN_MARIANA_ISLANDS') &
                                    (education_years['STATE'] != 'BI') &
                                    (education_years['STATE'] != 'DD') &
                                    (education_years['STATE'] != 'DOD_-_DOMESTIC') &
                                    (education_years['STATE'] != 'BIE') &
                                    (education_years['STATE'] != 'DOD_-_OVERSEAS') &
                                    (education_years['STATE'] != 'BUREAU_OF_INDIAN_EDUCATION') &
                                    (education_years['STATE'] != 'DEPARTMENT_OF_DEFENSE') &
                                    (education_years['STATE'] != 'DEPARTMENT_OF_DEFENSE_EDUCATION_ACTIVITY') &
                                    (education_years['STATE'] != 'BUREAU_OF_INDIAN_EDUCATIO') &
                                    (education_years['STATE'] != 'U.S._VIRGIN_ISLANDS') &
                                    (education_years['STATE'] != 'DOD_(OVERSEAS_AND_DOMESTIC_COMBINED)') &
                                    (education_years['STATE'] != 'DISTRICT_OF_COLUMBIA')]
updated_ed_df.head()


,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
459,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
460,ALASKA,2001,1402839.0,701053.0,10718.0,10702.0,124254.0,NaN,NaN,NaN,NaN
461,ARIZONA,2001,5858385.0,2538936.0,74270.0,69643.0,841750.0,NaN,NaN,NaN,NaN
462,ARKANSAS,2001,2951677.0,1616883.0,35216.0,35372.0,413278.0,NaN,NaN,NaN,NaN
463,CALIFORNIA,2001,52082540.0,26641811.0,485301.0,461133.0,5717489.0,NaN,NaN,NaN,NaN


#### Highlighting the issue with Virginia
* Three rows with different info

In [24]:
display(updated_ed_df['STATE'].value_counts().head()) # Virginia seems to have extra and is the only one
VA_ed = updated_ed_df.loc[(education_years['STATE'] == 'VIRGINIA')]
display(VA_ed['YEAR'].value_counts().head()) # 2008 seems to have Three observations
VA_data_error = VA_ed.loc[(VA_ed['YEAR'] == 2008)]
display(VA_data_error) # Unique Values in Enrollment Columns [4TH_ENROLLED, 8TH_ENROLLED, TOTAL_ENROLLMENT]

VIRGINIA     18
OKLAHOMA     16
MAINE        16
TENNESSEE    16
ARKANSAS     16
Name: STATE, dtype: int64

2008    3
2015    1
2014    1
2013    1
2012    1
Name: YEAR, dtype: int64

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
863,VIRGINIA,2008,15236306.0,8003167.0,4666.0,3863.0,51345.0,NaN,NaN,NaN,NaN
864,VIRGINIA,2008,15236306.0,8003167.0,2548.0,1485.0,24554.0,NaN,NaN,NaN,NaN
865,VIRGINIA,2008,15236306.0,8003167.0,91133.0,92881.0,1145864.0,NaN,NaN,NaN,NaN


In [25]:
ed_group_df = updated_ed_df.groupby(['STATE','YEAR']).mean()
ed_group_df.reset_index(inplace=True)
# ed_group_df['STATE'].value_counts() # Check to make sure we have the right number of obs
ed_group_df

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
0,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
1,ALABAMA,2002,5147821.0,2720735.0,58690.0,57576.0,683867.0,NaN,NaN,NaN,NaN
2,ALABAMA,2003,5298932.0,2817111.0,57323.0,59663.0,674679.0,223.344643,261.939022,207.083848,261.870872
3,ALABAMA,2004,5450269.0,2875595.0,56182.0,59286.0,673176.0,NaN,NaN,NaN,NaN
4,ALABAMA,2005,5904664.0,3053380.0,55919.0,59643.0,681201.0,225.071236,262.211100,207.752033,273.045757
5,ALABAMA,2006,6591429.0,3326656.0,56720.0,59573.0,685423.0,NaN,NaN,NaN,NaN
6,ALABAMA,2007,7196459.0,3653466.0,57795.0,58792.0,686947.0,228.520018,266.000110,216.388954,265.641982
7,ALABAMA,2008,7847387.0,3947003.0,58593.0,57809.0,689463.0,NaN,NaN,NaN,NaN
8,ALABAMA,2009,7815467.0,3836398.0,59512.0,57027.0,691870.0,227.962650,268.524120,216.273356,267.577474
9,ALABAMA,2010,7647571.0,3900492.0,59117.0,57992.0,697887.0,NaN,NaN,NaN,NaN


In [26]:
# Export to Reports

education_report = ed_group_df

education_report.to_csv('../Reports/education_report.csv')

education_report



,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
0,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
1,ALABAMA,2002,5147821.0,2720735.0,58690.0,57576.0,683867.0,NaN,NaN,NaN,NaN
2,ALABAMA,2003,5298932.0,2817111.0,57323.0,59663.0,674679.0,223.344643,261.939022,207.083848,261.870872
3,ALABAMA,2004,5450269.0,2875595.0,56182.0,59286.0,673176.0,NaN,NaN,NaN,NaN
4,ALABAMA,2005,5904664.0,3053380.0,55919.0,59643.0,681201.0,225.071236,262.211100,207.752033,273.045757
5,ALABAMA,2006,6591429.0,3326656.0,56720.0,59573.0,685423.0,NaN,NaN,NaN,NaN
6,ALABAMA,2007,7196459.0,3653466.0,57795.0,58792.0,686947.0,228.520018,266.000110,216.388954,265.641982
7,ALABAMA,2008,7847387.0,3947003.0,58593.0,57809.0,689463.0,NaN,NaN,NaN,NaN
8,ALABAMA,2009,7815467.0,3836398.0,59512.0,57027.0,691870.0,227.962650,268.524120,216.273356,267.577474
9,ALABAMA,2010,7647571.0,3900492.0,59117.0,57992.0,697887.0,NaN,NaN,NaN,NaN


# Joining the Cleaned Datasets

### Quick preview to remember what we've done

In [27]:
display('Cleaned Incarceration Info')
display(incarceration_report.head())
display('Cleaned Vera Info [we will make an SQL, but not join]')
display(vera_incarceration_report.head())
display('Cleaned Education Info')
display(education_report.head())



'Cleaned Incarceration Info'

,STATE,YEAR,PRISONER_COUNT,STATE_POPULATION,VIOLENT_CRIME_TOTAL,PROPERTY_CRIME_TOTAL
0,ALABAMA,2001,24741,4468912.0,19582.0,173253.0
1,ALABAMA,2002,25100,4478896.0,19931.0,180400.0
2,ALABAMA,2003,27614,4503726.0,19331.0,182241.0
3,ALABAMA,2004,25635,4525375.0,19324.0,182340.0
4,ALABAMA,2005,24315,4548327.0,19678.0,177393.0


'Cleaned Vera Info [we will make an SQL, but not join]'

,STATE,YEAR,TOTAL_JAIL_ADM,ASIAN_JAIL_POP,BLACK_JAIL_POP,LATINO_JAIL_POP,NATIVE_JAIL_POP,WHITE_JAIL_POP,ASIAN_PRISON_POP,BLACK_PRISON_POP,LATINO_PRISON_POP,NATIVE_PRISON_POP,OTHER_PRISON_POP,WHITE_PRISON_POP
0,ALABAMA,2001,179700.0,0.164394,52.606131,1.721304,0.135383,40.856784,0.0,63.212848,0.0,0.0,0.304336,36.482817
1,ALABAMA,2002,221681.0,0.297122,51.764160,2.293408,0.129991,44.113278,0.0,62.229226,0.0,0.0,0.221946,37.548828
2,ALABAMA,2003,158972.0,0.224417,41.303860,5.262567,0.112208,48.417864,0.0,53.034932,0.0,0.0,0.212512,46.752557
3,ALABAMA,2004,197394.0,0.297680,42.527202,4.034079,0.164237,47.207966,0.0,54.822695,0.0,0.0,0.153664,45.023641
4,ALABAMA,2005,194139.0,0.278124,42.476264,3.462166,0.143857,48.288098,0.0,53.184792,0.0,0.0,0.166646,46.648562


'Cleaned Education Info'

,STATE,YEAR,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,4TH_ENROLLED,8TH_ENROLLED,TOTAL_ENROLLMENT,4_AVG_MATH_SCORE,8_AVG_MATH_SCORE,4_AVG_RDG_SCORE,8_AVG_RDG_SCORE
0,ALABAMA,2001,5086045.0,2656393.0,58929.0,56591.0,683042.0,NaN,NaN,NaN,NaN
1,ALABAMA,2002,5147821.0,2720735.0,58690.0,57576.0,683867.0,NaN,NaN,NaN,NaN
2,ALABAMA,2003,5298932.0,2817111.0,57323.0,59663.0,674679.0,223.344643,261.939022,207.083848,261.870872
3,ALABAMA,2004,5450269.0,2875595.0,56182.0,59286.0,673176.0,NaN,NaN,NaN,NaN
4,ALABAMA,2005,5904664.0,3053380.0,55919.0,59643.0,681201.0,225.071236,262.211100,207.752033,273.045757


In [28]:
education_v_incarceration_format = education_v_incarceration
education_v_incarceration_format['TOTAL_EXPENDITURE'] = education_v_incarceration_format['TOTAL_EXPENDITURE']\
                                                        .map("${:,.2f}".format)
education_v_incarceration_format['INSTRUCTION_EXPENDITURE'] = education_v_incarceration_format['INSTRUCTION_EXPENDITURE']\
                                                        .map("${:,.2f}".format)
education_v_incarceration_format['4TH_ENROLLED'] = education_v_incarceration_format['4TH_ENROLLED']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['8TH_ENROLLED'] = education_v_incarceration_format['8TH_ENROLLED']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['TOTAL_ENROLLMENT'] = education_v_incarceration_format['TOTAL_ENROLLMENT']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['4_AVG_MATH_SCORE'] = education_v_incarceration_format['4_AVG_MATH_SCORE']\
                                                        .map("{:,.2f}".format)
education_v_incarceration_format['8_AVG_MATH_SCORE'] = education_v_incarceration_format['8_AVG_MATH_SCORE']\
                                                        .map("{:,.2f}".format)
education_v_incarceration_format['4_AVG_RDG_SCORE'] = education_v_incarceration_format['4_AVG_RDG_SCORE']\
                                                        .map("{:,.2f}".format)
education_v_incarceration_format['8_AVG_RDG_SCORE'] = education_v_incarceration_format['8_AVG_RDG_SCORE']\
                                                        .map("{:,.2f}".format)
education_v_incarceration_format['PRISONER_COUNT'] = education_v_incarceration_format['PRISONER_COUNT']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['STATE_POPULATION'] = education_v_incarceration_format['STATE_POPULATION']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['VIOLENT_CRIME_TOTAL'] = education_v_incarceration_format['VIOLENT_CRIME_TOTAL']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['PROPERTY_CRIME_TOTAL'] = education_v_incarceration_format['PROPERTY_CRIME_TOTAL']\
                                                        .map("{:,.0f}".format)
education_v_incarceration_format['ASIAN_JAIL_POP'] = education_v_incarceration_format['ASIAN_JAIL_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['BLACK_JAIL_POP'] = education_v_incarceration_format['BLACK_JAIL_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['LATINO_JAIL_POP'] = education_v_incarceration_format['LATINO_JAIL_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['NATIVE_JAIL_POP'] = education_v_incarceration_format['NATIVE_JAIL_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['WHITE_JAIL_POP'] = education_v_incarceration_format['WHITE_JAIL_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['ASIAN_PRISON_POP'] = education_v_incarceration_format['ASIAN_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['BLACK_PRISON_POP'] = education_v_incarceration_format['BLACK_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['LATINO_PRISON_POP'] = education_v_incarceration_format['LATINO_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['NATIVE_PRISON_POP'] = education_v_incarceration_format['NATIVE_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['OTHER_PRISON_POP'] = education_v_incarceration_format['OTHER_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format['WHITE_PRISON_POP'] = education_v_incarceration_format['WHITE_PRISON_POP'].map("{:,.2f}%".format)
education_v_incarceration_format.head()



NameError: name 'education_v_incarceration' is not defined

In [ ]:
ed_prison_df = education_report.merge(incarceration_report, on=['STATE', 'YEAR'])
education_v_incarceration = ed_prison_df.merge(vera_incarceration_report, on=['STATE', 'YEAR'])
education_v_incarceration.to_csv('../Reports/education_v_incarceration.csv')
display(education_v_incarceration.head(25))
display(education_v_incarceration.dtypes)

## Loading Output Reports

In [ ]:
education_v_incarceration_complete = education_v_incarceration_format[['STATE', 'YEAR', 'TOTAL_EXPENDITURE', 
                                 'INSTRUCTION_EXPENDITURE','4TH_ENROLLED', '8TH_ENROLLED', 'TOTAL_ENROLLMENT',
                                 'PRISONER_COUNT', 'STATE_POPULATION', 'VIOLENT_CRIME_TOTAL', 'PROPERTY_CRIME_TOTAL']]
education_v_incarceration_complete

In [ ]:
education_v_incarceration.to_csv('../Outputs/education_v_incarceration_raw.csv')
education_v_incarceration.to_json('../Outputs/education_v_incarceration_raw.json', orient='table')
education_v_incarceration_format.to_csv('../Outputs/education_v_incarceration_format.csv')
education_v_incarceration_format.to_json('../Outputs/education_v_incarceration_format.json', orient='table')
education_v_incarceration_complete.to_csv('../Outputs/education_v_incarceration_complete.csv')
education_v_incarceration_complete.to_json('../Outputs/education_v_incarceration_complete.json', orient='table')

## API Calls can be made

* Here via github
 
    https://raw.githubusercontent.com/katelynburke/educationvsincarceration/master/Outputs/education_v_incarceration_complete.json <br>
    https://raw.githubusercontent.com/katelynburke/educationvsincarceration/master/Outputs/education_v_incarceration_format.json <br>
    https://raw.githubusercontent.com/katelynburke/educationvsincarceration/master/Outputs/education_v_incarceration_raw.json <br><br>
            
* or Here via github pages

    https://srmonteiro.github.io/data/education_v_incarceration_complete.json <br>
    https://srmonteiro.github.io/data/education_v_incarceration_format.json <br>
    https://srmonteiro.github.io/data/education_v_incarceration_raw.json 
        

In [ ]:
engine = create_engine('sqlite://', echo=False)

In [ ]:
education_v_incarceration_format.to_sql(name='education_v_incarceration_format', con=engine, 
                                        if_exists = 'append', index=False)

